<center>
    This is Set 4: Frame Operations (Exercises 31-40)
</center>

**Prerequisites**
* The sample dataset [titanic](https://www.openml.org/d/40945) from openml will be used for the exercises.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F


In [2]:
# !wget https://www.openml.org/data/get_csv/16826755/titanic.csv


In [3]:
spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/25 16:30:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.read.options(header=True, inferSchema=True, nullValue="?").csv(
    "../data/titanic.csv"
)
data.show(5)


+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|pclass|survived|                name|   sex|   age|sibsp|parch|ticket|    fare|  cabin|embarked|boat|body|           home.dest|
+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|     1|       1|Allen, Miss. Elis...|female|  29.0|    0|    0| 24160|211.3375|     B5|       S|   2|null|        St Louis, MO|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|113781|  151.55|C22 C26|       S|  11|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Miss. He...|female|   2.0|    1|    2|113781|  151.55|C22 C26|       S|null|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Mr. Huds...|  male|  30.0|    1|    2|113781|  151.55|C22 C26|       S|null| 135|Montreal, PQ / Ch...|
|     1|       0|Allison, Mrs. Hud...|female|  25.0|    1|    2|113781|  151.55|C22 C26|       S|

**Exercise 31: Print all the column names and column types of `data` in `column-name : column-type` format**

In [26]:
data.dtypes


[('pclass', 'int'),
 ('survived', 'int'),
 ('name', 'string'),
 ('sex', 'string'),
 ('age', 'double'),
 ('sibsp', 'int'),
 ('parch', 'int'),
 ('ticket', 'string'),
 ('fare', 'double'),
 ('cabin', 'string'),
 ('embarked', 'string'),
 ('boat', 'string'),
 ('body', 'int'),
 ('home.dest', 'string')]

**Exercise 32: Convert the type of column `age` in `data` from `object` to `float`**

In [27]:
# Not needed, because during reading I've specified inferSchema=True and the type of the column is already double


**Exercise 33: Rename the column `home.dest` in `data` to `address`**

In [8]:
data = data.withColumnRenamed("home.dest", "address")
data.show(5)


+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|pclass|survived|                name|   sex|   age|sibsp|parch|ticket|    fare|  cabin|embarked|boat|body|             address|
+------+--------+--------------------+------+------+-----+-----+------+--------+-------+--------+----+----+--------------------+
|     1|       1|Allen, Miss. Elis...|female|  29.0|    0|    0| 24160|211.3375|     B5|       S|   2|null|        St Louis, MO|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|113781|  151.55|C22 C26|       S|  11|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Miss. He...|female|   2.0|    1|    2|113781|  151.55|C22 C26|       S|null|null|Montreal, PQ / Ch...|
|     1|       0|Allison, Mr. Huds...|  male|  30.0|    1|    2|113781|  151.55|C22 C26|       S|null| 135|Montreal, PQ / Ch...|
|     1|       0|Allison, Mrs. Hud...|female|  25.0|    1|    2|113781|  151.55|C22 C26|       S|

**Exercise 34: Sort `data` in descending order of `fare`**

In [9]:
data = data.sort("fare", ascending=False)
data.show(5)


+------+--------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+----+----+--------------------+
|pclass|survived|                name|   sex| age|sibsp|parch|  ticket|    fare|      cabin|embarked|boat|body|             address|
+------+--------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+----+----+--------------------+
|     1|       1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|   3|null|Austria-Hungary /...|
|     1|       1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|   3|null|                null|
|     1|       1|Cardeza, Mrs. Jam...|female|58.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|   3|null|Germantown, Phila...|
|     1|       1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|   3|null|                null|
|     1|       0|   Fortune, Mr. Mark|  male|64.0|    1|    4|   1995

**Exercise 35: Create a column `deck` in `data` containing only the first character of `cabin`**

In [10]:
data = data.withColumn("deck", data["cabin"].substr(0, 1))
data.select("deck").show(5)


+----+
|deck|
+----+
|   B|
|   B|
|   B|
|null|
|   C|
+----+
only showing top 5 rows



**Exercise 36: Create a column `family` in `data` which is the sum of two columns `sibsp` and `parch`**

In [11]:
data = data.withColumn("family", data["sibsp"] + data["parch"])
data.select("family").show(5)


+------+
|family|
+------+
|     1|
|     0|
|     1|
|     0|
|     5|
+------+
only showing top 5 rows



**Exercise 37: Create a column `total_passengers` in `data` which is the sum of `family` and the passenger included and round the values of column `age` in `data` to the nearest integer, both together using a single inplace operation**

In [12]:
data = data.withColumn("total_passengers", data["family"] + 1).withColumn(
    "age", F.round(data["age"])
)
data.show(5)


+------+--------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|pclass|survived|                name|   sex| age|sibsp|parch|  ticket|    fare|      cabin|embarked|boat|body|             address|deck|family|total_passengers|
+------+--------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|     1|       1|Cardeza, Mrs. Jam...|female|58.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|   3|null|Germantown, Phila...|   B|     1|               2|
|     1|       1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|   3|null|                null|   B|     0|               1|
|     1|       1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|   3|null|                null|null|     0|               1|
|     1|       1|Cardeza, Mr

**Exercise 38: Delete the used columns `sibsp` and `parch` and the outlier rows first and last from `data`**

In [13]:
data = data.drop(*["sibsp", "parch"])
data.show(5)


+------+--------+--------------------+------+----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|pclass|survived|                name|   sex| age|  ticket|    fare|      cabin|embarked|boat|body|             address|deck|family|total_passengers|
+------+--------+--------------------+------+----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|     1|       1|Cardeza, Mrs. Jam...|female|58.0|PC 17755|512.3292|B51 B53 B55|       C|   3|null|Germantown, Phila...|   B|     1|               2|
|     1|       1|Lesurer, Mr. Gust...|  male|35.0|PC 17755|512.3292|       B101|       C|   3|null|                null|   B|     0|               1|
|     1|       1|    Ward, Miss. Anna|female|35.0|PC 17755|512.3292|       null|       C|   3|null|                null|null|     0|               1|
|     1|       1|Cardeza, Mr. Thom...|  male|36.0|PC 17755|512.3292|B51 B53 B55|       C|   3|null|A

In [14]:
# PySpark DataFrames don't have indices, so we won't be able to easily define and delete first and last rows
data = spark.createDataFrame(data.rdd.collect()[1:-1])
data.show(5)


+------+--------+--------------------+------+----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|pclass|survived|                name|   sex| age|  ticket|    fare|      cabin|embarked|boat|body|             address|deck|family|total_passengers|
+------+--------+--------------------+------+----+--------+--------+-----------+--------+----+----+--------------------+----+------+----------------+
|     1|       1|Lesurer, Mr. Gust...|  male|35.0|PC 17755|512.3292|       B101|       C|   3|null|                null|   B|     0|               1|
|     1|       1|    Ward, Miss. Anna|female|35.0|PC 17755|512.3292|       null|       C|   3|null|                null|null|     0|               1|
|     1|       1|Cardeza, Mr. Thom...|  male|36.0|PC 17755|512.3292|B51 B53 B55|       C|   3|null|Austria-Hungary /...|   B|     1|               2|
|     1|       1|Fortune, Miss. Al...|female|24.0|   19950|   263.0|C23 C25 C27|       S|  10|null| 

**Exercise 39: Replace all `'?'` values with `NA` or `null` as missing values in `data`**

In [15]:
# already done while reading the file


**Exercise 40: Impute all the missing values in `data` with `-1` for numeric columns and `'unknown'` for string columns**

In [19]:
(
    data.select(
        *(F.sum(F.col(c).isNull().cast("int")).alias(c) for c in data.columns)
    ).show()
)


+------+--------+----+---+---+------+----+-----+--------+----+----+-------+----+------+----------------+
|pclass|survived|name|sex|age|ticket|fare|cabin|embarked|boat|body|address|deck|family|total_passengers|
+------+--------+----+---+---+------+----+-----+--------+----+----+-------+----+------+----------------+
|     0|       0|   0|  0|263|     0|   0| 1013|       2| 822|1187|    563|1013|     0|               0|
+------+--------+----+---+---+------+----+-----+--------+----+----+-------+----+------+----------------+



In [23]:
data = data.na.fill("unknown").na.fill(-1)
data.show(10)


+------+--------+--------------------+------+----+--------+--------+-----------+--------+-------+----+--------------------+-------+------+----------------+
|pclass|survived|                name|   sex| age|  ticket|    fare|      cabin|embarked|   boat|body|             address|   deck|family|total_passengers|
+------+--------+--------------------+------+----+--------+--------+-----------+--------+-------+----+--------------------+-------+------+----------------+
|     1|       1|Lesurer, Mr. Gust...|  male|35.0|PC 17755|512.3292|       B101|       C|      3|  -1|             unknown|      B|     0|               1|
|     1|       1|    Ward, Miss. Anna|female|35.0|PC 17755|512.3292|    unknown|       C|      3|  -1|             unknown|unknown|     0|               1|
|     1|       1|Cardeza, Mr. Thom...|  male|36.0|PC 17755|512.3292|B51 B53 B55|       C|      3|  -1|Austria-Hungary /...|      B|     1|               2|
|     1|       1|Fortune, Miss. Al...|female|24.0|   19950|   26

✅ This completes Set 4: Frame Operations (Exercises 31-40)

Original exercises for Datatable package can be found [here](https://github.com/vopani/datatableton).

Pandas version can be found [there](https://github.com/Extremesarova/pandas_exercises).